In [3]:
import numpy as np
import pandas as pd
import os

import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pyampute
import pickle 
import time

from scipy.stats import mstats
from scipy.stats.mstats import winsorize
from scipy import stats
from xgboost import XGBRegressor
from sklearn import tree
from pyampute.ampute import MultivariateAmputation
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from lifelines import CoxPHFitter, WeibullFitter, WeibullAFTFitter
from datetime import datetime, date, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tableone import TableOne 
from os.path import isfile, join
from sklearn.metrics import mean_absolute_error, roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from statsmodels.gam.tests.test_penalized import df_autos
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import norm
from scipy.spatial import distance
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from app_transition_dict import get_transition_dict
import warnings 
warnings.filterwarnings('ignore')


drive = 'G'
main_path = drive + ':/Shared drives/CKD_Progression/data/CKD_COHORT_Jan2010_Mar2024_v3.csv'
data_path = drive + ':/Shared drives/CKD_Progression/data/'
docs_path = drive + ':/Shared drives/CKD_Progression/docs/'
save_path = drive + ':/Shared drives/CKD_Progression/save/'
resu_path = drive + ':/Shared drives/CKD_Progression/result/'
covariates_path = docs_path + 'covariates.csv'
removecols_path = docs_path + 'remove_columns.csv'

def calculate_summary_statistics(df):
    mean_values   = df.mean()
    std_values    = df.std()
    median_values = df.median()
    iqr_values    = df.quantile(0.75) - df.quantile(0.25)
    summary_stats = pd.DataFrame({
        'Mean': mean_values,
        'Standard Deviation': std_values,
        'Median': median_values,
        'IQR': iqr_values})
    return summary_stats

In [11]:
columns_impute = ['BMI', 'BW', 'HIGH', 'Lipid_HDL', 'Lipid_LDL', 'Chem_glucose', 'Chem_HbA1C', 'Renal_Uric_acid', 
                  'Lipid_Cholesterol', 'Renal_Serum_creatinine', 'Lipid_Triglyceride', 'age', 'Renal_eGFR', 'CVD']

main_data = pd.read_csv(save_path + 'qoc_cohort_ver002.csv')
main_data = main_data.rename(columns = {'height': 'HIGH'})
main_data = main_data[columns_impute]
main_data['BMI'] = np.divide(main_data['BW'], np.divide(main_data['HIGH'], 100) ** 2)
calculate_summary_statistics(main_data).to_csv(resu_path + 'imputation/models/LinearReg/statistics_orig.csv')

In [18]:
pd.DataFrame((main_data.isnull().sum()/main_data.shape[0])*100).to_csv(resu_path + 'imputation/models/LinearReg/missing_rate.csv')

In [4]:
for dataset in ['90']:
    df = pd.read_csv(resu_path + f'imputation/LinearReg/IMPUTED_LinearReg_{dataset}D_10I.csv')
    df['BMI'] = np.divide(df['BW'], np.divide(df['HIGH'], 100) ** 2)
    calculate_summary_statistics(df).to_csv(resu_path + f'imputation/models/LinearReg/statistics_resu_{dataset}D.csv')